In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load pre-trained models and scalers
# try:
#     model_si = joblib.load('model_si.pkl')
#     model_tandem_top = joblib.load('model_tandem_top.pkl')
#     model_tandem_bot = joblib.load('model_tandem_bot.pkl')
#     scaler_si = joblib.load('scaler_si.pkl')
#     scaler_tandem = joblib.load('scaler_tandem.pkl')
#     print("✅ Models loaded successfully!")
# except:
#     print("❌ Could not load models. Please ensure model files exist.")
#     # Create dummy models for demonstration
#     model_si = RandomForestRegressor()
#     model_tandem_top = RandomForestRegressor()
#     model_tandem_bot = RandomForestRegressor()
#     scaler_si = StandardScaler()
#     scaler_tandem = StandardScaler()

model_si = joblib.load('model_si.pkl')
model_tandem_top = joblib.load('model_tandem_top.pkl')
model_tandem_bot = joblib.load('model_tandem_bot.pkl')
scaler_si = joblib.load('scaler_si.pkl')
scaler_tandem = joblib.load('scaler_tandem.pkl')
print("✅ Models loaded successfully!")

✅ Models loaded successfully!


In [3]:
def create_pointillism_image(image_path, opacity_level):
    """Create pointillism image with specified opacity level"""
    import cv2
    import numpy as np
    import matplotlib.pyplot as plt
    
    # Parameters (matching the original notebook)
    dot_density = 4
    dot_scale = 1.0
    alpha_value = opacity_level
    use_adaptive_sizing = True
    
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"{image_path} not found.")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    h, w, _ = img.shape
    
    # Optional cropping (adjust as needed)
    img_c = img[145:145+1080, :]
    h_c, w_c, _ = img_c.shape
    
    # Resize to control dot density
    img_r = cv2.resize(img_c, (w_c // dot_density, h_c // dot_density), interpolation=cv2.INTER_AREA)
    h_r, w_r, _ = img_r.shape
    
    # Convert to grayscale for brightness
    gray = cv2.cvtColor(img_r, cv2.COLOR_RGB2GRAY)
    
    # Adaptive sizing
    if use_adaptive_sizing:
        edges = cv2.Canny(gray, 50, 150)
        edges = cv2.GaussianBlur(edges, (5, 5), 0)
        size_map = 0.3 + (edges / 255.0) * 0.7
    else:
        size_map = np.ones_like(gray, dtype=float)
    
    # Build scatter data
    lX, lY, lS, lC = [], [], [], []
    
    for i in range(h_r):
        for j in range(w_r):
            lX.append(i)
            lY.append(j)
            
            brightness = gray[i, j]
            relative_brightness = brightness / 255.0
            size = (1.0 - relative_brightness) * 0.8 + 0.2
            
            size *= size_map[i, j]
            size *= dot_scale
            
            lS.append(size)
            lC.append(img_r[i, j] / 255.0)
    
    # Convert to arrays
    lX = np.array(lX)
    lY = np.array(lY)
    lS = np.array(lS)
    lC = np.array(lC)
    
    # Normalize sizes
    lS = 50 + (lS / lS.max()) * 150
    
    # Create plot
    fig = plt.figure(figsize=(60, 60), dpi=100, facecolor='none')
    ax = plt.subplot(111)
    ax.patch.set_alpha(0)
    
    scatter = ax.scatter(
        lY, lX,
        s=lS,
        c=lC,
        alpha=alpha_value,
        edgecolors='none',
        marker='o',
        rasterized=True
    )
    
    ax.set_aspect('equal')
    ax.axis('off')
    ax.invert_yaxis()
    plt.tight_layout(pad=0)
    
    # Convert plot to image array
    fig.canvas.draw()
    plot_array = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    plot_array = plot_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)
    
    return plot_array

In [4]:
def predict_current_from_array(image_array, model_type="si"):
    """Predict current density from image array"""
    # Convert array to PIL Image and then to RGBA
    img = Image.fromarray(image_array).convert("RGBA")
    img_array = np.array(img)
    h, w, _ = img_array.shape
    
    # Reshape into (n_pixels, 4)
    pixels = img_array.reshape(-1, 4)
    pixel_df = pd.DataFrame(pixels, columns=['R', 'G', 'B', 'A'])
    
    # Predict
    if model_type.lower() == "si":
        X_scaled = scaler_si.transform(pixel_df)
        jph_preds = model_si.predict(X_scaled)
        return np.mean(jph_preds)
    
    elif model_type.lower() == "tandem":
        X_scaled = scaler_tandem.transform(pixel_df)
        jph_top = model_tandem_top.predict(X_scaled)
        jph_bot = model_tandem_bot.predict(X_scaled)
        jph_limiting = np.minimum(jph_top, jph_bot)
        return np.mean(jph_limiting)
    
    else:
        raise ValueError("Invalid model_type. Use 'si' or 'tandem'.")

In [5]:
def evolutionary_algorithm_optimization(image_path, model_type="si", generations=10, population_size=8):
    """Evolutionary Algorithm to find optimal opacity level"""
    print("🚀 Starting Evolutionary Algorithm for opacity optimization...")
    
    # Initialize population
    population = np.random.uniform(0.1, 0.9, population_size)
    best_opacity = 0.5
    best_current = 0
    
    # Store results for analysis
    results = []
    
    for generation in range(generations):
        print(f"\nGeneration {generation + 1}/{generations}")
        generation_results = []
        
        for i, opacity in enumerate(population):
            try:
                # Create pointillism image with current opacity
                pointillism_img = create_pointillism_image(image_path, opacity)
                
                # Predict current
                current = predict_current_from_array(pointillism_img, model_type)
                generation_results.append((opacity, current))
                
                print(f"  Individual {i+1}: opacity={opacity:.3f}, current={current:.3f} mA/cm²")
                
                # Update best solution
                if current > best_current:
                    best_current = current
                    best_opacity = opacity
                    
            except Exception as e:
                print(f"  Error with opacity {opacity}: {e}")
                generation_results.append((opacity, 0))
        
        results.extend(generation_results)
        
        # Selection: Keep best individuals
        generation_results.sort(key=lambda x: x[1], reverse=True)
        best_individuals = generation_results[:population_size//2]
        
        # Crossover and Mutation
        new_population = [ind[0] for ind in best_individuals]  # Elitism
        
        while len(new_population) < population_size:
            # Select parents from best individuals
            parent1, parent2 = np.random.choice([ind[0] for ind in best_individuals], 2, replace=False)
            
            # Crossover (average)
            child = (parent1 + parent2) / 2
            
            # Mutation
            mutation = np.random.normal(0, 0.1)
            child = np.clip(child + mutation, 0.1, 0.9)
            
            new_population.append(child)
        
        population = np.array(new_population)
    
    # Find opacity with 20% loss from max current
    target_current = best_current * 0.8
    optimal_opacity = best_opacity
    min_diff = float('inf')
    
    for opacity, current in results:
        diff = abs(current - target_current)
        if diff < min_diff and current >= target_current:
            min_diff = diff
            optimal_opacity = opacity
    
    print(f"\n🎯 Optimization Complete!")
    print(f"Best opacity: {best_opacity:.3f} (Current: {best_current:.3f} mA/cm²)")
    print(f"Optimal opacity with 20% loss: {optimal_opacity:.3f}")
    print(f"Target current: {target_current:.3f} mA/cm²")
    
    return optimal_opacity, best_current, results

In [6]:
def split_image_into_cells(image_array, n_cells):
    """Split image into n x n grid of solar cells"""
    h, w, _ = image_array.shape
    
    # Calculate grid dimensions
    cells_per_side = int(np.sqrt(n_cells))
    if cells_per_side ** 2 != n_cells:
        print(f"⚠️ Warning: {n_cells} is not a perfect square. Using {cells_per_side**2} cells instead.")
        n_cells = cells_per_side ** 2
    
    cell_height = h // cells_per_side
    cell_width = w // cells_per_side
    
    cells = []
    positions = []
    
    for i in range(cells_per_side):
        for j in range(cells_per_side):
            y_start = i * cell_height
            y_end = (i + 1) * cell_height
            x_start = j * cell_width
            x_end = (j + 1) * cell_width
            
            cell = image_array[y_start:y_end, x_start:x_end]
            cells.append(cell)
            positions.append((i, j))
    
    return cells, positions, cells_per_side

In [7]:
def analyze_solar_cells(image_path, optimal_opacity, n_cells, model_type="si"):
    """Analyze current density for each solar cell"""
    print(f"\n🔬 Analyzing {n_cells} solar cells with opacity {optimal_opacity:.3f}...")
    
    # Create pointillism image with optimal opacity
    pointillism_img = create_pointillism_image(image_path, optimal_opacity)
    
    # Split into cells
    cells, positions, grid_size = split_image_into_cells(pointillism_img, n_cells)
    
    # Calculate current density for each cell
    cell_currents = []
    for i, cell in enumerate(cells):
        current = predict_current_from_array(cell, model_type)
        cell_currents.append(current)
        print(f"  Cell {i+1} (row {positions[i][0]}, col {positions[i][1]}): {current:.3f} mA/cm²")
    
    # Create current density matrix
    current_matrix = np.zeros((grid_size, grid_size))
    for (row, col), current in zip(positions, cell_currents):
        current_matrix[row, col] = current
    
    return cell_currents, current_matrix, grid_size, pointillism_img

In [8]:
def equalize_currents(cell_currents, method="lowest"):
    """Equalize currents to match the specified method"""
    if method == "lowest":
        target_current = min(cell_currents)
    elif method == "average":
        target_current = np.mean(cell_currents)
    else:
        raise ValueError("Method must be 'lowest' or 'average'")
    
    equalized_currents = [target_current] * len(cell_currents)
    
    print(f"\n⚡ Current equalization ({method} method):")
    print(f"Target current: {target_current:.3f} mA/cm²")
    print(f"Original current range: {min(cell_currents):.3f} - {max(cell_currents):.3f} mA/cm²")
    print(f"Equalized all cells to: {target_current:.3f} mA/cm²")
    
    return equalized_currents, target_current

In [9]:
# Main execution
if __name__ == "__main__":
    # Configuration
    IMAGE_PATH = "CJ.png"  # Replace with your image path
    MODEL_TYPE = "si"  # "si" or "tandem"
    N_CELLS = 9  # Number of solar cells (should be perfect square: 4, 9, 16, 25, etc.)
    
    print("=== Solar Cell Current Density Analysis ===")
    print(f"Image: {IMAGE_PATH}")
    print(f"Model: {MODEL_TYPE}")
    print(f"Number of cells: {N_CELLS}")
    
    # Step 1: Find optimal opacity using Evolutionary Algorithm
    print("\n" + "="*50)
    optimal_opacity, max_current, ea_results = evolutionary_algorithm_optimization(
        IMAGE_PATH, MODEL_TYPE, generations=8, population_size=6
    )
    
    # Step 2: Split image into solar cells and analyze current densities
    print("\n" + "="*50)
    cell_currents, current_matrix, grid_size, processed_img = analyze_solar_cells(
        IMAGE_PATH, optimal_opacity, N_CELLS, MODEL_TYPE
    )
    
    # Step 3: Equalize currents to match the lowest cell
    print("\n" + "="*50)
    equalized_currents, target_current = equalize_currents(cell_currents, method="lowest")
    
    # Visualization
    print("\n" + "="*50)
    print("📊 Generating visualizations...")
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Plot 1: Original pointillism image
    axes[0, 0].imshow(processed_img)
    axes[0, 0].set_title(f'Pointillism Image (Opacity: {optimal_opacity:.3f})')
    axes[0, 0].axis('off')
    
    # Plot 2: Current density heatmap
    im = axes[0, 1].imshow(current_matrix, cmap='plasma', aspect='equal')
    axes[0, 1].set_title('Current Density Heatmap (mA/cm²)')
    axes[0, 1].set_xlabel('Column')
    axes[0, 1].set_ylabel('Row')
    
    # Add values to heatmap
    for i in range(grid_size):
        for j in range(grid_size):
            axes[0, 1].text(j, i, f'{current_matrix[i, j]:.2f}', 
                           ha='center', va='center', color='white', fontweight='bold')
    
    plt.colorbar(im, ax=axes[0, 1])
    
    # Plot 3: Original vs Equalized currents
    x_pos = np.arange(len(cell_currents))
    width = 0.35
    
    axes[1, 0].bar(x_pos - width/2, cell_currents, width, label='Original', alpha=0.7)
    axes[1, 0].bar(x_pos + width/2, equalized_currents, width, label='Equalized', alpha=0.7)
    axes[1, 0].axhline(y=target_current, color='r', linestyle='--', label=f'Target: {target_current:.3f} mA/cm²')
    axes[1, 0].set_xlabel('Solar Cell Index')
    axes[1, 0].set_ylabel('Current Density (mA/cm²)')
    axes[1, 0].set_title('Original vs Equalized Currents')
    axes[1, 0].legend()
    axes[1, 0].set_xticks(x_pos)
    
    # Plot 4: EA optimization progress
    opacities, currents = zip(*ea_results)
    axes[1, 1].scatter(opacities, currents, alpha=0.6, c=currents, cmap='viridis')
    axes[1, 1].axvline(x=optimal_opacity, color='red', linestyle='--', label=f'Optimal: {optimal_opacity:.3f}')
    axes[1, 1].set_xlabel('Opacity Level')
    axes[1, 1].set_ylabel('Current Density (mA/cm²)')
    axes[1, 1].set_title('EA Optimization Results')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print("\n" + "="*50)
    print("📈 SUMMARY STATISTICS")
    print(f"Optimal opacity level: {optimal_opacity:.3f}")
    print(f"Maximum current density: {max_current:.3f} mA/cm²")
    print(f"Target current (20% loss): {max_current * 0.8:.3f} mA/cm²")
    print(f"Number of solar cells: {N_CELLS}")
    print(f"Grid configuration: {grid_size} × {grid_size}")
    print(f"Original current statistics:")
    print(f"  - Mean: {np.mean(cell_currents):.3f} mA/cm²")
    print(f"  - Std: {np.std(cell_currents):.3f} mA/cm²")
    print(f"  - Min: {np.min(cell_currents):.3f} mA/cm²")
    print(f"  - Max: {np.max(cell_currents):.3f} mA/cm²")
    print(f"Equalized current: {target_current:.3f} mA/cm²")
    
    # Save results
    results_df = pd.DataFrame({
        'Cell_Index': range(1, len(cell_currents) + 1),
        'Row': [pos[0] for pos in positions],
        'Column': [pos[1] for pos in positions],
        'Original_Current': cell_currents,
        'Equalized_Current': equalized_currents
    })
    
    results_df.to_csv('solar_cell_analysis_results.csv', index=False)
    print(f"\n💾 Results saved to 'solar_cell_analysis_results.csv'")

=== Solar Cell Current Density Analysis ===
Image: CJ.png
Model: si
Number of cells: 9

🚀 Starting Evolutionary Algorithm for opacity optimization...

Generation 1/8
  Individual 1: opacity=0.189, current=11.883 mA/cm²
  Individual 2: opacity=0.546, current=11.878 mA/cm²
  Individual 3: opacity=0.626, current=11.878 mA/cm²
  Individual 4: opacity=0.202, current=11.883 mA/cm²
  Individual 5: opacity=0.350, current=11.881 mA/cm²
  Individual 6: opacity=0.390, current=11.881 mA/cm²

Generation 2/8
  Individual 1: opacity=0.189, current=11.883 mA/cm²
  Individual 2: opacity=0.202, current=11.883 mA/cm²
  Individual 3: opacity=0.350, current=11.881 mA/cm²
  Individual 4: opacity=0.261, current=11.881 mA/cm²
  Individual 5: opacity=0.226, current=11.883 mA/cm²
  Individual 6: opacity=0.358, current=11.881 mA/cm²

Generation 3/8
  Individual 1: opacity=0.189, current=11.883 mA/cm²
  Individual 2: opacity=0.202, current=11.883 mA/cm²
  Individual 3: opacity=0.226, current=11.883 mA/cm²
  Indiv

KeyboardInterrupt: 